<a href="https://colab.research.google.com/github/duotien/mono_pitch_tracker/blob/main/mono_pitch_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!nvidia-smi

Tue Feb 28 11:39:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
%%capture
!git clone --recursive https://github.com/duotien/mono_pitch_tracker.git
!apt -qq install -y sox
!pip install pyyaml==5.4.1 sox librosa==0.9.2

In [ ]:
%%capture
!unzip -o V1_MIX_AUDIO_ONLY.zip -d /content/mono_pitch_tracker/medleydb/medleydb/data/Audio/

In [ ]:
import sys
import os

NOTEBOOK_DIR = os.getcwd()
MONO_PITCH_TRACKER_DIR = os.path.join(os.getcwd(), 'mono_pitch_tracker/')
MEDLEYDB_DIR = os.path.join(MONO_PITCH_TRACKER_DIR, 'medleydb/')

sys.path.append(MONO_PITCH_TRACKER_DIR)
sys.path.append(MEDLEYDB_DIR)

In [ ]:
# change working directory to `mono_pitch_tracker/``
%cd $MONO_PITCH_TRACKER_DIR

# Prepare Dataset

In [ ]:
# cut audio into frames
import numpy as np
import pandas as pd

import medleydb
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH,
                                           N_CLASS, )
from pitch_tracker.utils.audio import load_audio_mono
from pitch_tracker.utils.files import get_file_name, list_file_paths_in_dir, list_folder_paths_in_dir

# from pitch_tracker.utils import dataset, files
from pitch_tracker.utils.medleydb_melody import gen_label
from pitch_tracker.utils import dataset

/content/mono_pitch_tracker/medleydb/medleydb/__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/content/mono_pitch_tracker/medleydb/medleydb/__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


In [ ]:
# Prepare datset
!python scripts/prepare_dataset.py

# Modeling & training

In [ ]:
# Train model
! python scripts/model.py

In [ ]:
model = torch.load('/content/drive/MyDrive/School work/KLTN/2022-2023/models/mpt_v01_20231028-2010.pth')

In [ ]:
import torch

dummy_in = torch.randn(8, 1, 1050, 88)
in_names = [ "actual_input_1" ] + [ "learned_%d" % i for i in range(16) ]
out_names = [ "output1" ]

torch.onnx.export(model, dummy_in, f="model.onnx", input_names=in_names, output_names=out_names, opset_version=7, verbose=True)